# Detectron2 Inference - Organoid Detection with IoU Evaluation

This notebook demonstrates using a trained Detectron2 model to:
1. Detect organoids in microscopy images
2. Evaluate predictions against ground truth masks using IoU metric
3. Export results with visualizations and IoU scores

**Prerequisites:** A trained Detectron2 model (from the training notebook)

## 1. Installation and Setup

In [1]:
# Install dependencies
!python -m pip install pyyaml==5.1

# Import standard libraries
import sys
import os
import distutils.core
import torch
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
from PIL import Image
from google.colab.patches import cv2_imshow

# Install Detectron2
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Import Detectron2 components
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 11.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15943, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15943 (delta 4), reused 3 (delta 3), pack-reused 15930 (from 2)
Receiving objects: 100% (15943/15943), 6.70 MiB | 12.20 MiB/s, done.
Resolving deltas: 100% (11336/11336), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your envir

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Load Trained Model

In [4]:
# Copy trained model from Google Drive
!cp -r "/content/drive/My Drive/Detectron2_Organoid_FineTuning/02_Model" /content

In [5]:
# Configure model for inference
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# Dataset configuration
cfg.DATASETS.TRAIN = ("organoid_train",)
cfg.DATASETS.TEST = ()

# Model configuration
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Single class: organoid

# Load trained weights
cfg.MODEL.WEIGHTS = os.path.join("/content/02_Model", "model_final.pth")

# Inference threshold
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

# Create predictor
predictor = DefaultPredictor(cfg)

print("✓ Model loaded successfully")

✓ Model loaded successfully


## 3. Inference on New Images with IoU Evaluation

### Configuration

Define input/output paths and image processing parameters.

In [6]:
# Input/Output paths
input_folder = '/content/drive/My Drive/Detectron2_Organoid_FineTuning/03_Data/Evaluation_Test/Inputs'
output_folder = '/content/drive/My Drive/Detectron2_Organoid_FineTuning/03_Data/Evaluation_Test/Masks_Output'
mask_folder = '/content/drive/My Drive/Detectron2_Organoid_FineTuning/03_Data/Evaluation_Test/Ground_Truth'
visual_output_folder = '/content/drive/My Drive/Detectron2_Organoid_FineTuning/03_Data/Evaluation_Test/Visual_Results'
excel_path = '/content/drive/My Drive/Detectron2_Organoid_FineTuning/04_Metrics_and_Results/IoU_Scores.xlsx'

# Image resize parameters
max_width = 520
max_height = 693

# Create output directories
for folder in [output_folder, visual_output_folder]:
    os.makedirs(folder, exist_ok=True)

print("✓ Configuration complete")

✓ Configuration complete


### IoU Calculation Function

In [7]:
def calculate_iou(pred_mask, true_mask):
    """
    Calculate Intersection over Union (IoU) between predicted and ground truth masks.

    Args:
        pred_mask: Predicted binary mask
        true_mask: Ground truth binary mask

    Returns:
        IoU score (float between 0 and 1)
    """
    pred_mask = pred_mask.astype(bool)
    true_mask = true_mask.astype(bool)

    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()

    if union == 0:
        return 0.0

    return intersection / union

### Process Images and Calculate IoU

In [8]:
import pandas as pd

# Initialize results storage
results = []

# Get list of images to process
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg'))]

print(f"Found {len(image_files)} images to process\n")

# Process each image
for filename in tqdm(image_files, desc="Processing images"):
    # Define paths
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".jpg")
    mask_path = os.path.join(mask_folder, os.path.splitext(filename)[0] + ".jpg")
    visual_path = os.path.join(visual_output_folder, filename)

    # Skip if ground truth mask doesn't exist
    if not os.path.exists(mask_path):
        continue

    # Read and resize input image
    im = cv2.imread(input_path)
    if im is None:
        continue

    height, width = im.shape[:2]
    ratio = min(max_width / width, max_height / height)
    new_width, new_height = int(width * ratio), int(height * ratio)
    im = cv2.resize(im, (new_width, new_height))

    # Run prediction
    outputs = predictor(im)

    # Save visualization with predictions
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                   scale=1.0)
    out_visual = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(visual_path, out_visual.get_image()[:, :, ::-1])

    # Load and process ground truth mask
    true_mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    true_mask_img = cv2.resize(true_mask_img, (new_width, new_height), interpolation=cv2.INTER_NEAREST)
    true_mask_binary = (true_mask_img > 128).astype(np.uint8)

    # Extract predicted mask
    pred_mask_binary = np.zeros((new_height, new_width), dtype=np.uint8)

    if len(outputs["instances"]) > 0:
        masks = outputs["instances"].pred_masks.cpu().numpy()
        scores = outputs["instances"].scores.cpu().numpy()
        # Select mask with highest confidence score
        max_score_idx = np.argmax(scores)
        pred_mask_binary = masks[max_score_idx].astype(np.uint8)

    # Save predicted mask as image
    mask_image = np.ones((new_height, new_width, 3), dtype=np.uint8) * 255
    mask_image[pred_mask_binary == 0] = 1
    cv2.imwrite(output_path, mask_image)

    # Calculate IoU
    iou_score = calculate_iou(pred_mask_binary, true_mask_binary)

    # Store results
    results.append({
        'Image': os.path.splitext(filename)[0],
        'IoU Score': iou_score
    })

print("\n✓ Processing complete!")

Found 6 images to process



Processing images:   0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W0208 15:38:56.707000 220 torch/fx/_symbolic_trace.py:52] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.
Processing images: 100%|██████████| 6/6 [00:12<00:00,  2.08s/it]


✓ Processing complete!


### Save Results and Display Summary

In [9]:
# Save results to Excel
if results:
    df = pd.DataFrame(results)
    df.to_excel(excel_path, index=False)

    print(f"✓ IoU scores saved to: {excel_path}")
    print(f"✓ Visual outputs saved to: {visual_output_folder}")
    print(f"✓ Predicted masks saved to: {output_folder}")

    # Display summary statistics
    print("\n=== Results Summary ===")
    print(f"Total images processed: {len(df)}")
    print(f"\nIoU Score Statistics:")
    print(df['IoU Score'].describe())

    # Display first few results
    print("\n=== Sample Results ===")
    display(df.head(10))
else:
    print("No results to save.")

print("\n✓ Processing complete!")

✓ IoU scores saved to: /content/drive/My Drive/Detectron2_Organoid_FineTuning/04_Metrics_and_Results/IoU_Scores.xlsx
✓ Visual outputs saved to: /content/drive/My Drive/Detectron2_Organoid_FineTuning/03_Data/Evaluation_Test/Visual_Results
✓ Predicted masks saved to: /content/drive/My Drive/Detectron2_Organoid_FineTuning/03_Data/Evaluation_Test/Masks_Output

=== Results Summary ===
Total images processed: 6

IoU Score Statistics:
count    6.000000
mean     0.833813
std      0.154378
min      0.532003
25%      0.834340
50%      0.889505
75%      0.915649
max      0.951553
Name: IoU Score, dtype: float64

=== Sample Results ===


,Image,IoU Score
0,999,0.951553
1,1089,0.532003
2,994,0.867350
3,1029,0.916978
4,997,0.823336
5,987,0.911660



✓ Processing complete!
